# Ćwiczenie 3

Celem ćwiczenia jest imlementacja metody [Minimax z obcinaniem alpha-beta](https://en.wikipedia.org/wiki/Alpha%E2%80%93beta_pruning) do gry Connect Four (czwórki).

W trakcie ćwiczenia można skorzystać z reposytorium z implementacją gry [Connect Four udostępnionym przez Jakuba Łyskawę](https://github.com/lychanl/two-player-games). Ewentualnie, można zaimplementować samemu grę Connect Four (ale, tak aby rozwiązanie miało ten sam interfejs co podany poniżej).

Implementację Minimax należy przetestować używając różną głębokość przeszukiwania. Implementacja Solvera musi zapewniać interfejs jak poniżej, ale można dodać dowolne metody prywatne oraz klasy wspomagające (jeżeli będą potrzebne).

Punktacja:
- Działająca metoda Minimax - **2 pkt**
- Działająca metoda Minimax z obcinaniem alpha-beta - **1.5 pkt**
- Analiza jakości solvera w zależności od głębokości przeszukiwania **1.5pkt**
    - należy zaimplementować w tym celu prostą wizualizację rozgrywki dwóch agentów, bądź kilka przykładów 'z ręki'
- Jakość kodu **2pkt**

Aby importowanie elementów z poniższej komórki działało należy umieścić tego notebooka w tym samym folderze co paczkę `two_player_games`:
```
├── LICENSE
├── README.md
├── minimax.ipynb # <<< HERE
├── test
│   ├── __init__.py
│   ├── test_connect_four.py
│   ├── test_dots_and_boxes.py
│   └── test_pick.py
└── two_player_games
    ├── __init__.py
    ├── games
    │   ├── connect_four.py
    │   └── dots_and_boxes.py
    ├── move.py
    ├── player.py
    └── state.py
```

In [30]:
from typing import Tuple, List
from math import inf

from two_player_games.player import Player
from two_player_games.games.connect_four import ConnectFour, ConnectFourMove

Wielkość planszy

In [31]:
ROW_COUNT = 6
COLUMN_COUNT = 7

In [32]:
class MinMaxSolver:

    def __init__(self, game: ConnectFour):
        self.game = game

    def evaluate_position(self, player: Player)->float:
        score = 0
        board = self.game.state.fields

        # pionowo
        for column in range(COLUMN_COUNT):
            temp_column = [board[i][1] for i in range(ROW_COUNT)]
            for row in range(3):
                check_area = temp_column[row:row+4]
                score += self.evaluate_area(check_area, player)

        # poziomo
        for row in range(ROW_COUNT):
            for column in range(4):
                check_area = board[row][column:column+4]
                score += self.evaluate_area(check_area, player)
        
        # po skosie
        for row in range(ROW_COUNT-3):
            for column in range(COLUMN_COUNT-3):
                check_area = [board[column+i][row+i] for i in range(4)]
                score += self.evaluate_area(check_area, player)
                check_area = [board[column+3-i][row+i] for i in range(4)]
                score += self.evaluate_area(check_area, player)

        return score
    
    def evaluate_area(self, check_area, player):
        score = 0
        if player == self.game.first_player:
            other_player = self.game.second_player
        else:
            other_player = self.game.first_player
        
        if check_area.count(player) == 4:
            score += 200
        elif check_area.count(player) == 3 and check_area.count(None) == 1:
            score += 30
        elif check_area.count(player) == 2 and check_area.count(None) == 2:
            score += 5
        if check_area.count(other_player) == 4:
            score -= 200
        elif check_area.count(other_player) == 3 and check_area.count(None) == 1:
            score -= 30
        elif check_area.count(other_player) == 2 and check_area.count(None) == 2:
            score -= 5
        return score

    def get_best_move(self)->int:
        pass

    def is_valid_move(self, col_index:int)->bool:
        return self.game.state.fields[col_index][ROW_COUNT-1] == None
    
    def minimax(self, depth, alpha:float, beta:float, is_maximizing_player:bool)-> Tuple[int, float]:
        if depth == 0 or self.game.get_winner() is not None:
            if self.game.get_winner() is not None:
                if self.game.get_winner() == self.game.get_current_player():
                    return (None, inf)
                elif self.game.get_winner() != self.game.get_current_player():
                    return (None, -inf)
                else:
                    return (None, 0)
            else:
                return (None, self.evaluate_position(self.game.get_current_player()))
        if is_maximizing_player:
            score = -inf
            for column in range(COLUMN_COUNT):
                if self.is_valid_move(column):
                    temp_game = self.game.kopiuj()
                    temp_game.make_move(ConnectFourMove(column))
                    temp_solver = MinMaxSolver(temp_game)
                    temp_score = temp_solver.minimax(depth-1, alpha, beta, False)[1]
                    if temp_score > score:
                        score = temp_score
                        best_move = column
        else:
            score = inf
            for column in range(COLUMN_COUNT):
                if self.is_valid_move(column):
                    temp_game = self.game.kopiuj()
                    temp_game.make_move(ConnectFourMove(column))
                    temp_solver = MinMaxSolver(temp_game)
                    temp_score = temp_solver.minimax(depth-1, alpha, beta, True)[1]
                    if temp_score < score:
                        score = temp_score
                        best_move = column
        
        """Returns column index and score"""
        return (best_move, score)

Rozgrywka

In [33]:
p1 = Player("a")
p2 = Player("b")
game = ConnectFour(size=(COLUMN_COUNT, ROW_COUNT), first_player=p1, second_player=p2)
solver = MinMaxSolver(game)
i=0
while not game.is_finished():
    move = solver.minimax(3, 0, 0, True)
    game.make_move(ConnectFourMove(move[0]))
    print(game)
    i+=1
print(i)
winner = game.get_winner()
if winner is None:
    print('Draw!')
else:
    print('Winner: Player ' + winner.char)


AttributeError: 'ConnectFour' object has no attribute 'kopiuj'

In [ ]:
p1 = Player("a")
p2 = Player("b")
game = ConnectFour(size=(COLUMN_COUNT, ROW_COUNT), first_player=p1, second_player=p2)

# game.make_move(ConnectFourMove(0))
# game.make_move(ConnectFourMove(5))
# game.make_move(ConnectFourMove(0))
# game.make_move(ConnectFourMove(5))
# game.make_move(ConnectFourMove(0))
# game.make_move(ConnectFourMove(5))
# game.make_move(ConnectFourMove(0))
# game.make_move(ConnectFourMove(5))
i=0
while not game.is_finished():
    if i%2==0:
        game.make_move(ConnectFourMove(0))
    else:
        game.make_move(ConnectFourMove(1))
    i +=1
    print(game)

# moves = game.state.get_moves()
# fields = game.state.fields
# temp = fields[0:4][0]
# temp2 = fields[0][0:4]
# print(game)
# print(temp)
# print(temp2)

winner = game.get_winner()
if winner is None:
    print('Draw!')
else:
    print('Winner: Player ' + winner.char)

# print(game)

Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[a][ ][ ][ ][ ][ ][ ]
Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[a][b][ ][ ][ ][ ][ ]
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[a][ ][ ][ ][ ][ ][ ]
[a][b][ ][ ][ ][ ][ ]
Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[a][b][ ][ ][ ][ ][ ]
[a][b][ ][ ][ ][ ][ ]
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[a][ ][ ][ ][ ][ ][ ]
[a][b][ ][ ][ ][ ][ ]
[a][b][ ][ ][ ][ ][ ]
Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[a][b][ ][ ][ ][ ][ ]
[a][b][ ][ ][ ][ ][ ]
[a][b][ ][ ][ ][ ][ ]
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[a][ ][ ][ ][ ][ ][ ]
[a][b][ ][ ][ ][